## Loading Libraries

In [ ]:
using ModelingToolkit
using GLMakie
using ModelingToolkit: t_nounits as t, D_nounits as D
using OrdinaryDiffEq

In [ ]:

# Simulation settings


@parameters α β A
@variables M(t)
tspan = (0.0, 50.0)
timesteps = 0.0:0.1:50.0
α_val = 1.0
β_val = 0.1
A_val = 1.0
reinforce_interval = 10.0
noise_fraction = 0.2
width = 0.5


## No_Reinforcement

In [ ]:
eqs1 = [D(M) ~ - β*M]
@mtkbuild memsys1 = ODESystem(eqs1, t)
u0 = [M => 1.0]
p1 = [α => α_val, β => β_val, A => A_val]
prob1 = ODEProblem(memsys1, u0, tspan, p1)
sol1 = solve(prob1, Tsit5(); saveat=timesteps)

fig1 = Figure(resolution=(600,400))
ax1 = Axis(fig1[1,1], xlabel="Time", ylabel="Memory Strength", title="Case 1: No Reinforcement (The Cram)")
lines!(ax1, sol1.t, sol1[1,:], color=:red, linewidth=3)
display(fig1)

## Periodic Reinforcement

In [ ]:
# Smooth pulse for periodic reinforcement
function smooth_pulse(t, interval, width, A)
    sum(A * exp(-(t - n*interval)^2 / (2*width^2)) for n in 0:ceil(Int, tspan[2]/interval))
end

eqs2 = [D(M) ~ α * smooth_pulse(t, reinforce_interval, width, A) - β*M]
@mtkbuild memsys2 = ODESystem(eqs2, t)
p2 = [α => α_val, β => β_val, A => A_val]
prob2 = ODEProblem(memsys2, u0, tspan, p2)
sol2 = solve(prob2, Tsit5(); saveat=timesteps)

fig2 = Figure(resolution=(600,400))
ax2 = Axis(fig2[1,1], xlabel="Time", ylabel="Memory Strength", title="Case 2: Periodic Reinforcement (Spaced Repetition)")
lines!(ax2, sol2.t, sol2[1,:], color=:green, linewidth=3)
display(fig2)

## Reinforcement with Noise

In [ ]:
pulse_noisy = smooth_pulse(t, reinforce_interval, width*0.5, A*0.7)     

eqs3 = [D(M) ~ α * pulse_noisy - β*M]
@mtkbuild memsys3 = ODESystem(eqs3, t)
p3 = [α => α_val, β => β_val, A => A_val]
prob3 = ODEProblem(memsys3, u0, tspan, p3)
sol3 = solve(prob3, Tsit5(); saveat=timesteps)

fig3 = Figure(resolution=(600,400))
ax3 = Axis(fig3[1,1], xlabel="Time", ylabel="Memory Strength", title="Case 3: Reinforcement + Noise (Reduced Strength)")
lines!(ax3, sol3.t, sol3[1,:], color=:blue, linewidth=3)
display(fig3)

## Visualization

In [ ]:
fig = Figure(size=(1000, 800), fontsize=14, backgroundcolor=:white)


# 2x2 Grid of Axes

ax1 = Axis(fig[2,1], xlabel="Time", ylabel="Memory Strength", title="Case 1: No Reinforcement (The Cram)")
ax2 = Axis(fig[2,2], xlabel="Time", ylabel="Memory Strength", title="Case 2: Periodic Reinforcement (Spaced Repetition)")
ax3 = Axis(fig[3,1], xlabel="Time", ylabel="Memory Strength", title="Case 3: Reinforcement + Noise (Real-World Learning)")
ax4 = Axis(fig[3,2], xlabel="Time", ylabel="Memory Strength", title="Overlay Comparison")


# Plot Lines

l1 = lines!(ax1, sol1.t, sol1[1,:], color=:red, linewidth=2.5)
l2 = lines!(ax2, sol2.t, sol2[1,:], color=:green, linewidth=2.5)
l3 = lines!(ax3, sol3.t, sol3[1,:], color=:blue, linewidth=2.5)

# Overlay plot
l4 = lines!(ax4, sol1.t, sol1[1,:], color=:red, linewidth=2.5, label="No Reinforcement")
l5 = lines!(ax4, sol2.t, sol2[1,:], color=:green, linewidth=2.5, label="Periodic Reinforcement")
l6 = lines!(ax4, sol3.t, sol3[1,:], color=:blue, linewidth=2.5, linestyle=:dash, label="Reinforcement + Noise")


# Common Legend at the top

common_legend = Legend(fig, [l4, l5, l6], ["No Reinforcement", "Periodic Reinforcement", "Reinforcement + Noise"],
                       framevisible=false, orientation=:horizontal, halign=:center)

fig[1,1:2] = common_legend


# Auto-fit all axes

for ax in (ax1, ax2, ax3, ax4)
    autolimits!(ax)
end

display(fig)
GLMakie.save("Memory_Dynamics.png", fig)


In [ ]:



# Figure setup

fig = Figure(size=(1000, 800), fontsize=14, backgroundcolor=:white)


ax1 = Axis(fig[2,1], xlabel="Time", ylabel="Memory Strength", title="No Reinforcement (The Cram)")
ax2 = Axis(fig[2,2], xlabel="Time", ylabel="Memory Strength", title="Periodic Reinforcement (Spaced Repetition)")
ax3 = Axis(fig[3,1], xlabel="Time", ylabel="Memory Strength", title="Noisy Reinforcement (Real-World Learning)")
ax4 = Axis(fig[3,2], xlabel="Time", ylabel="Memory Strength", title="Overlay Comparison")


# Plot static curves

l1 = lines!(ax1, sol1.t, sol1[1,:], color=:red, linewidth=2.5)
l2 = lines!(ax2, sol2.t, sol2[1,:], color=:green, linewidth=2.5)
l3 = lines!(ax3, sol3.t, sol3[1,:], color=:blue, linewidth=2.5)

l4 = lines!(ax4, sol1.t, sol1[1,:], color=:red, linewidth=2.5, label="No Reinforcement")
l5 = lines!(ax4, sol2.t, sol2[1,:], color=:green, linewidth=2.5, label="Periodic Reinforcement")
l6 = lines!(ax4, sol3.t, sol3[1,:], color=:blue, linewidth=2.5, linestyle=:dash, label="Reinforcement + Noise")


# Common legend at top

common_legend = Legend(fig, [l4, l5, l6], ["No Reinforcement", "Periodic Reinforcement", "Reinforcement + Noise"],
                       framevisible=false, orientation=:horizontal, halign=:center)
fig[1,1:2] = common_legend 


# Animated dots

frame_idx = Observable(1)
dots1 = scatter!(ax1, [sol1.t[1]], [sol1[1,1]], color=:black, markersize=8)
dots2 = scatter!(ax2, [sol2.t[1]], [sol2[1,1]], color=:black, markersize=8)
dots3 = scatter!(ax3, [sol3.t[1]], [sol3[1,1]], color=:black, markersize=8)
dots4a = scatter!(ax4, [sol1.t[1]], [sol1[1,1]], color=:black, markersize=8)
dots4b = scatter!(ax4, [sol2.t[1]], [sol2[1,1]], color=:orange, markersize=8)
dots4c = scatter!(ax4, [sol3.t[1]], [sol3[1,1]], color=:purple, markersize=8)

on(frame_idx) do i
    i = clamp(i, 1, length(sol1.t))
    dots1[1][] = [Point2f(sol1.t[i], sol1[1,i])]
    dots2[1][] = [Point2f(sol2.t[i], sol2[1,i])]
    dots3[1][] = [Point2f(sol3.t[i], sol3[1,i])]
    dots4a[1][] = [Point2f(sol1.t[i], sol1[1,i])]
    dots4b[1][] = [Point2f(sol2.t[i], sol2[1,i])]
    dots4c[1][] = [Point2f(sol3.t[i], sol3[1,i])]
end


# Record animation

record(fig, "Memory_Dynamics.mp4", 1:length(sol1.t); framerate=20) do i
    frame_idx[] = i
end
